# Save and load models

Model có thể được save trong khi hoặc sau khi training. Điều này có lợi cho việc tránh phải training trong thời gian dài và chia sẻ mô hình đã training cho người khác. 

### Options

Có nhiều cách khác nhau để lưu các mô hình TensorFlow, tùy thuộc vào API bạn đang sử dụng. Hướng dẫn này sử dụng tf.keras. 

## Setup

### Installs and imports

Install and import TensorFlow and dependencies:

In [1]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

!pip install -q pyyaml h5py  # Required to save models in HDF5 format

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


### Get an example dataset

To demonstrate how to save and load weights, you'll use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). To speed up these runs, use the first 1000 examples:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Define a model

Start by building a simple sequential model:

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Save checkpoints during training

Bạn có thể sử dụng mô hình đã được đào tạo mà không cần phải đào tạo lại hoặc trong trường hợp quá trình đào tạo xảy ra gián đoạn, mô hình sẽ tiếp tục được đào tạo tại điểm gián đoạn đó. The `tf.keras.callbacks.ModelCheckpoint` callback cho phép lưu mô hình trong cả khi **during training** hoặc **the end of training**.

### Checkpoint callback usage

Create a `tf.keras.callbacks.ModelCheckpoint` callback thế lưu mô hình khi **during training**:

In [5]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Tạo callback để lưu trọng số của model
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model sử dụng callback vừa tạo
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Truyền trường callback để training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
 928/1000 [==========================>...] - ETA: 0s - loss: 1.2334 - accuracy: 0.6379
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 1s 1ms/sample - loss: 1.1896 - accuracy: 0.6530 - val_loss: 0.7383 - val_accuracy: 0.7740
Epoch 2/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.4359 - accuracy: 0.8683
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 362us/sample - loss: 0.4423 - accuracy: 0.8630 - val_loss: 0.5677 - val_accuracy: 0.8220
Epoch 3/10
 928/1000 [==========================>...] - ETA: 0s - loss: 0.2886 - accuracy: 0.9159
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 311us/sample - loss: 0.2983 - accuracy: 0.9160 - val_loss: 0.4609 - val_accuracy: 0.8480
Epoch 4/10
 896/1000 [=========================>....] - ETA: 0s - loss: 0.2065 - accuracy: 0.9442
Epoch 0

In [6]:
!ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


Tạo một mô hình mới chưa được đào tạo. Khi khôi phục một mô hình chỉ từ trọng số, bạn phải có một mô hình có cùng kiến trúc với mô hình ban đầu. Nếu mô hình có cùng kiến trúc, bạn có thể chia sẻ trọng số có chu đó là một hình thức model khác. 

Bây giờ xây dựng lại một mô hình mới chưa được đào tạo và đánh giá nó trên bộ thử nghiệm. Một mô hình chưa được đào tạo sẽ có độ chính xác ~ 10%:


In [7]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.2127 - accuracy: 0.1130
Untrained model, accuracy: 11.30%


Then load the weights from the checkpoint and re-evaluate:

In [8]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4397 - accuracy: 0.8700
Restored model, accuracy: 87.00%


### Checkpoint callback options
Callback cung cấp những lựa chọn cho việc cung cấp tên cụ thể cho mỗi checkpoints và điều chỉnh tần số của checkpointing.
Đào tạo một model mới, và lưu tên duy nhất cho mỗi checkpoint cho mỗi 5 epochs 1 lần :

In [9]:
# Thêm thành phần epoch trong file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Tạo callback và lưu lại trọng số của model đối với mỗi 5 epoch
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
              train_labels,
              epochs=50, 
              callbacks=[cp_callback],
              validation_data=(test_images,test_labels),
              verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


Now, look at the resulting checkpoints and choose the latest one:

In [10]:
!ls {checkpoint_dir}

checkpoint			  cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index		  cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index		  cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index		  cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index		  cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index		  cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [11]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

Note: mặc định của tensorflow chỉ lưu 5 checkpoint gần nhất.

To test, reset the model and load the latest checkpoint:

In [12]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5902 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## What are these files?

Filé chứa các trọng số được đào tạo ở định dạng nhị phân. Checkpoints chứa:
* Một hoặc nhiều phân đoạn có chứa trọng số của mô hình.
* Một index file cho biết trọng số nào được lưu trong phân đoạn nào.

Nếu bạn chỉ đào tạo một mô hình trên một máy duy nhất,bạn sẽ có một phân đoạn với hậu tố: `.data-00000-of-00001`

## Manually save weights
Cách save model đơn giản với phương thức `Model.save_weights`. 
Mặc định, tf.keras và save_weights nói riêng sử dụng định dạng Checkpoint với đuôi .ckpt()

In [13]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.5902 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## Lưu toàn booj model

Gọi model.save để lưu cấu trúc, trọng số và cấu hình đào tạo của model trong một file/folder. Điều này cho phép bạn xuất một model để nó có thể được sử dụng mà không cần sử dụng code Python. Bạn có thể tiếp tục đào tạo từ chính xác nơi bạn rời khỏi trong quá trình training.

### HDF5 format

Keras cung cấp định dạng cơ bản bằng cách sử dụng tiêu chuẩn HDF5.

In [14]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 521us/sample - loss: 1.1438 - accuracy: 0.6790
Epoch 2/5
1000/1000 [==============================] - 0s 171us/sample - loss: 0.4207 - accuracy: 0.8850
Epoch 3/5
1000/1000 [==============================] - 0s 175us/sample - loss: 0.2808 - accuracy: 0.9260
Epoch 4/5
1000/1000 [==============================] - 0s 185us/sample - loss: 0.2083 - accuracy: 0.9480
Epoch 5/5
1000/1000 [==============================] - 0s 192us/sample - loss: 0.1629 - accuracy: 0.9660


Now, recreate the model from that file:

In [15]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


Check its accuracy:

In [16]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

1000/1 - 0s - loss: 0.5496 - accuracy: 0.8750
Restored model, accuracy: 87.50%


Kỹ thuật này giúp chúng ta lưu mọi thứ:

* Trọng số
* Cấu trúc của mô hình
* Cấu hình optimizer

Keras lưu model bằng cách kiểm tra kiến trúc. Hiện tại, nó không thể lưu Tensorflow optimizers(from `tf.train`). Muốn sử dụng chúng thì cần phải re-compile lại model sau khi loading, và khi đó sẽ bị mất trạng thái hiện tại của optimizer.


### SavedModel format

The SavedModel format là một cách khác để tuần tự hóa các models. Các mô hình được lưu ở định dạng này có thể được khôi phục bằng `tf.keras.models.load_model`. Phần bên dưới minh họa các bước để save và restoring mô hình.

In [17]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 608us/sample - loss: 1.1571 - accuracy: 0.6770
Epoch 2/5
1000/1000 [==============================] - 0s 177us/sample - loss: 0.4204 - accuracy: 0.8780
Epoch 3/5
1000/1000 [==============================] - 0s 185us/sample - loss: 0.2840 - accuracy: 0.9240
Epoch 4/5
1000/1000 [==============================] - 0s 177us/sample - loss: 0.2068 - accuracy: 0.9490
Epoch 5/5
1000/1000 [==============================] - 0s 177us/sample - loss: 0.1469 - accuracy: 0.9680
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


The SavedModel format is a directory containing a protobuf binary and a Tensorflow checkpoint. Inspect the saved model directory:

In [18]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets	saved_model.pb	variables


Reload a fresh Keras model from the saved model:

In [20]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


The restored model is compiled with the same arguments as the original model. Try running evaluate and predict with the loaded model:

In [21]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

1000/1 - 0s - loss: 0.4462 - accuracy: 0.8530
Restored model, accuracy: 85.30%
(1000, 10)


### Saving custom objects

Sự khác biệt chính giữa HDF5 và SavingModel là HDF5 sử dụng các **object configs** để lưu kiến trúc model, trong khi SavingModel lưu **execution graph**(biểu đồ thực thi). Do đó, SavingModels có thể lưu các **object configs** như các **subclassed models** và các **custom layers** mà không yêu cầu mã nguồn.

Để lưu các cấu hình object trong HDF5, cần những bước sau:

1. Định nghĩa một phương thức `get_config` trong đối tượng của bạn và tùy chọn một lớp đối xứng `from_config`.
  * `get_config (self)` trả về một từ điển JSON-serializable các tham số cần thiết để tạo lại đối tượng.
  * `from_config(cls, config)` sử dụng cấu hình được trả về từ `get_config` để tạo một đối tượng mới. Mặc định, chức năng này sẽ sử dụng cấu hình làm kwargs khởi tạo (return cls (** config)).
2. Truyền đối tượng cho đối số custom_objects khi tải mô hình. Đối số phải là một từ điển ánh xạ tên lớp chuỗi sang lớp Python. E.g. `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`

See the [Writing layers and models from scratch](https://www.tensorflow.org/guide/keras/custom_layers_and_models) tutorial for examples of custom objects and `get_config`.
